In [3]:
import time
import datetime
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import log_loss
from sklearn.cross_validation import train_test_split

D:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
def timestamp_datetime(value):
    format = '%Y-%m-%d %H:%M:%S'
    value = time.localtime(value)
    dt = time.strftime(format, value)
    dt = datetime.datetime.strptime(dt, format)
    return dt

In [6]:
def convert_data(data):
    data['time'] = data.context_timestamp.apply(timestamp_datetime)
    data['day'] = data.time.apply(lambda x: x.day)
    data['hour'] = data.time.apply(lambda x: x.hour)
    user_query_day = data.groupby(['user_id', 'day']).size().reset_index().rename(columns={0: 'user_query_day'})
    data = pd.merge(data, user_query_day, 'left', on=['user_id', 'day'])
    user_query_day_hour = data.groupby(['user_id', 'day', 'hour']).size().reset_index()\
                            .rename(columns={0: 'user_query_day_hour'})
    data = pd.merge(data, user_query_day_hour, 'left', on=['user_id', 'day', 'hour'])
    print(data.head())
    return data

In [8]:
if __name__ == "__main__":
    online = True# 这里用来标记是 线下验证 还是 在线提交

    data = pd.read_csv('./data/round1_ijcai_18_train_20180301.txt', sep=' ')
    data.drop_duplicates(inplace=True)
    data = convert_data(data)

    if online == False:
#         data = data.sample(frac=1, random_state = 1)
#         train = data[:int(len(data) * 0.9)]  
#         test = data[int(len(data) * 0.9): ]  
          
        train = data.loc[data.day < 24]    # 18,19,20,21,22,23,24
        test = data.loc[data.day == 24]    # 暂时先使用第24天作为验证集
    elif online == True:
        train = data.copy()
        test = pd.read_csv('./data/round1_ijcai_18_test_a_20180301.txt', sep=' ')
        test = convert_data(test)
    
#             item_id: 广告商品编号，Long类型 ;
#             item_category_list: 广告商品的的类目列表;
#             item_property_list: 广告商品的属性列表
#             item_brand_id: 广告商品的品牌编号
                
                
    features = ['item_id', 'item_brand_id', 'item_city_id', 'item_price_level', 'item_sales_level',
                'item_collected_level', 'item_pv_level', 'user_gender_id', 'user_occupation_id',
                'user_age_level', 'user_star_level', 'user_query_day', 'user_query_day_hour',
                'context_page_id', 'hour', 'shop_id', 'shop_review_num_level', 'shop_star_level',
                'shop_review_positive_rate', 'shop_score_service', 'shop_score_delivery', 'shop_score_description',
                ]
    target = ['is_trade']
    
    if online == False:
        clf = lgb.LGBMClassifier(num_leaves=65, max_depth=5, n_estimators=100, n_jobs=20)
        clf.fit(train[features], train[target], feature_name=features,
                categorical_feature=['user_gender_id'])
        test['lgb_predict'] = clf.predict_proba(test[features],)[:, 1]
        print(log_loss(test[target], test['lgb_predict']))
    else:
        clf = lgb.LGBMClassifier(num_leaves=65, max_depth=5, n_estimators=100, n_jobs=20)
        clf.fit(train[features], train[target],
                categorical_feature=['user_gender_id'])
        test['predicted_score'] = clf.predict_proba(test[features])[:, 1]
        test[['instance_id', 'predicted_score']].to_csv('result4_5.csv', index=False,sep=' ')#保存在线提交结果

SyntaxError: invalid syntax (<ipython-input-8-06d498afee71>, line 20)